## Read earnings reports

E. Quinn 12/22/2019

This notebook uses pdfminer to extract the information from the individual earnings report

The documentation for pdfminer is at:

https://buildmedia.readthedocs.org/media/pdf/pdfminer-docs/latest/pdfminer-docs.pdf

## Import standard python datascience packages

In [ ]:
import math
import re
import copy
import numpy as np
import scipy as sc
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
from datetime import *
from datascience import *

## Import pdfminer packages

In [ ]:
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfpage import PDFTextExtractionNotAllowed
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfdevice import PDFDevice
from pdfminer.layout import LAParams
from pdfminer.converter import PDFPageAggregator
from pdfminer.pdfpage import PDFPage
from pdfminer.layout import LTTextBoxHorizontal

### Show the directory we are running in

In [ ]:
!pwd

## Read the pdf and create a dictionary with the contents of each text box

### Function read_earnings() reads the earnings report and returns a dictionary containing the contents

Strategy for this document:  

Save information from each element in the LTTextBox objects in a dictionary including:

- x0 horizontal coordinate of the upper left corner of the text box
- x1 horizontal coordinate of the lower right corner of the text box
- y0 vertical coordinate of the upper left corner of the text box
- y1 vertical coordinate of the lower right corner of the text box
- page number 
- sequence number of text box within this page
- text contained in the text box, converted to ascii

Parsing the text is complicated by the fact that that a text box may span multiple columns and/or rows, and the text box groupings vary quite a bit depending on the page contents and layout.

However, with a bit of luck the structure of the document will allow the contents to be deciphered with the following heuristics:

- Text boxes containing left justified columns will tend to have nearly the same x0 coordinates
- Text boxes containing right justified columns will tend to have nearly the same x1 coordinates
- The codes for fund, account code, and object code are numeric and have fixed lengths
- Extraneous information is often preceded or followed by a series of underscore and newline characters
- Last name can be distinguished because is the only field that is all characters followed by a comma
- Last name may be preceded by between one and three numerical fields:  fund, account, object.  If it is, the x0 value is shifted to the left.
    - Three numerical fields precede the name:  assume they are fund, account, object
    - Two numerical fields precede the name: assume they are account, object
    - One numerical field precedes the name: assume it is object
    

In [ ]:
def read_earnings(path):
    document = open(path, 'rb')                                     #read a pdf and create a document object
    rsrcmgr = PDFResourceManager()                                  #create a resource manager
    laparams = LAParams()                                           #set the parameters for analysis
    device = PDFPageAggregator(rsrcmgr, laparams=laparams)          #create a PDF page aggregator object
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    
    pdf={}                                                          #dictionary to hold the results

    pageno = -1                                                     #initialize page coounter to zero

    for page in PDFPage.get_pages(document):                        #loop through the pdf page by page
        pageno = pageno + 1                                         #increment the page number
        pdf[pageno] = {}                                            #dictionary for this page
        interpreter.process_page(page)                              # receive the LTPage object for the page.
        layout = device.get_result()                                # create layout object
        tbox_no=0                                                   # index for element number
        for element in layout:
            if (type(element).__name__=='LTTextBoxHorizontal'):     #loop through text boxes
                tbox_no += 1                                        #increment text box number
                pdf[pageno][tbox_no] = {}                           #dictionary for text boxes within page
                x0 = round(element.x0,2)                            #x0 coordinate of textbox corner
                x1 = round(element.x1,2)                            #x1 coordinate of textbox corner
                y0 = round(element.y0,2)                            #y0 coordinate of textbox corner
                y1 = round(element.y1,2)                            #y1 coordinate of textbox corner
                txt = element.get_text().encode('ascii', 'ignore')  #text converted to ascii
                pdf[pageno][tbox_no]['x0'] = x0                     #create x0 coordinate entry
                pdf[pageno][tbox_no]['x1'] = x1                     #create x1 coordinate entry
                pdf[pageno][tbox_no]['y0'] = y0                     #create y0 coordinate entry
                pdf[pageno][tbox_no]['y1'] = y1                     #create y1 coordinate entry

                pdf[pageno][tbox_no]['text'] = ''.join(chr(c) for c in txt) #convert bytes to string
    return(pdf)

### Utility functions

In [ ]:
#remove the commas from earnings and rate values

def remove_commas(st):
    newstr = st.replace(',','')                     #remove commas from string
    return(newstr)

In [ ]:
#remove the headings fields 

def remove_headings(st):
    lines = st.split('\n')                         #split the string at newline characters '\n'
    for line in lines:                             #loop through the resulting lines
        if (line.startswith('FUND ') |\
           (line.startswith('POSITION')) |\
           (line.startswith('RATE')) |\
           (line.startswith('ACCT-')) |\
           (line.startswith('_'))):                #check for strings that appear only in headings
            try:
                newline_index = st.index('\n')     #if present, remove this line from the text string
                st = st[newline_index+1:]
            except ValueError:
                print('Value Error',st)            #recover from Value Error and print string
        else:
            return(st)                             #if no headings, just return
    return('')

### Read labels for accounting codes

In [ ]:
acct_df = pd.read_csv("../earnings_acct_codes_2017_UCOA.csv")
acct_dct = acct_df.to_dict()

accd = {}

for i in acct_dct['acct-code'].keys():
    acct = str(acct_dct['acct-code'][i])
    if (acct not in accd.keys()):
        accd[acct] = acct_dct['desc'][i]

### Build a dictionary with only those text boxes containing names

Use the following algorithm to identify text boxes that contain names:

- x0, horizontal coordinate of the upper left corner of the text box, is less than 162
- the text string contains at least one comma

In [ ]:
def get_names(dct):

    dnames = {}

    fund = ''
    acct = ''
    obj  = ''
    
    for page in sorted(dct.keys()):                                #loop through text box dictionary by page # 
        if (page not in dnames.keys()):                            #page number is highest level key
            dnames[page] = {}                                      #initialize entry for this page
        for tb in sorted(dct[page].keys()):                        #loop through all text boxes on this page
            if (dct[page][tb]['x0'] < 162.0):                      #those with names start to the left of x0=162
                txt = str(dct[page][tb]['text'])                   #convert the 'text' element to a string
                if (',' in txt):                                   #every name contains a comma
                    txt = remove_headings(txt)
                    lines = txt.split('\n')                        #split text into lines
                    words = lines[0].split()                       #split first line into words
                    for word in words:                             #loop through and strip out fund, acct, obj
                        if (word.isdigit()):
                            if (len(word)==4):                     # 4 digits means fund
                                fund = word
                            if (len(word)==8):                     # 8 digits means acct-code
                                acct = word
                            if (len(word)==5):                     # 5 digits means obj
                                obj = word
                            txt = txt[len(word)+1:]                # remove fund/acct/obj from txt
                    dnames[page][tb] = {}                          #initialize dictionary for this page
                    dnames[page][tb]['x0'] = dct[page][tb]['x0']
                    dnames[page][tb]['x1'] = dct[page][tb]['x1']
                    dnames[page][tb]['y0'] = dct[page][tb]['y0']
                    dnames[page][tb]['y1'] = dct[page][tb]['y1']
                    dnames[page][tb]['fund'] = fund
                    dnames[page][tb]['acct'] = acct
                    dnames[page][tb]['obj'] = obj
                    dnames[page][tb]['text'] = txt
    return(dnames)

### Consolidate text boxes that overlap on the vertical scale and contain names

In [ ]:
def consolidate_name_boxes(names):
    newnames = {}
    
    for page in sorted(names.keys()):                                        #loop through pages of pdf
        newnames[page] = {}                                                  #initialize new names dictionary
        skip = make_array()                                                  #initialize list of boxes to skip
    
        for tb in sorted(names[page].keys()):                                #loop through text boxes on this page
            for tb2 in sorted(names[page].keys()):                           #compare this one to the others
                if ((tb2 > tb) & \
                    (names[page][tb]['y0'] <= names[page][tb2]['y1']) & \
                    (names[page][tb2]['y0'] <= names[page][tb]['y1'])):      
                    d = {}                                                   #initialize replacement entry
                    d['x0'] = names[page][tb]['x0']                          #keep x0    
                    d['x1'] = names[page][tb2]['x1']                         #replace x1 with tb2 value
                    d['y0'] = names[page][tb2]['y0']                         #replace y0 with tb2 value
                    d['y1'] = names[page][tb]['y1']                          #keep y1 value
                    d['text'] = names[page][tb]['text'] +\
                        names[page][tb2]['text']                             #contatenate text strings
                    d['fund'] = names[page][tb]['fund']                      #copy fund, acct, and obj
                    d['acct'] = names[page][tb]['acct']
                    d['obj'] = names[page][tb]['obj']
                    newnames[page][tb2] = d                                  #plug into dictionary
                    skip = np.append(skip,tb)                                #add old boxes to skip list
                    skip = np.append(skip,tb2)
            if (tb not in skip):                                             #if no match, check skip list 
                newnames[page][tb] = names[page][tb]                         #just copy if not in skip list
                    
    return(newnames)

### Match consolidated text boxes with names to text boxes for other columns

In [ ]:
def get_matches(names,pdf):

    for page in sorted(names.keys()):                                       #loop through pages
        for tb in sorted(names[page].keys()):                               #loop through text boxes
            y0 = names[page][tb]['y0']                                      #extract vertical coordinates
            y1 = names[page][tb]['y1']
            txt = names[page][tb]['text']                                   #extract text
           
            for tb2 in sorted(pdf[page].keys()):                            #loop through the other boxes in pdf
                if (tb != tb2):                                             #ignore if same box as names
                    tx0 = pdf[page][tb2]['x0']                              #get horizontal offset
                    ty0 = pdf[page][tb2]['y0']                              #check whether the vertical 
                    ty1 = pdf[page][tb2]['y1']                              #range of this box overlaps that
                    if ((y0 <= ty1) & (ty0 <= y1)):                         #of the name box
                        txt = remove_headings(pdf[page][tb2]['text'])
                        if ((437.0 < tx0) & (tx0 < 440.0)):                 #match to POSITION
                            names[page][tb]['positions'] = txt
                        if ((509.0 < tx0) & (tx0 < 533.0)):                 #match to RATE 
                            names[page][tb]['rates'] = remove_commas(txt)
                        if ((558.0 < tx0) & (tx0 < 630.0)):                 #match to ACCT-EARNINGS
                            names[page][tb]['earnings'] = remove_commas(txt)
    return                                                              

### Get sum of earnings lines

Used for balancing to total on earnings report

In [ ]:
def balancing_totals(dct):
    
    total=0.0                                                       #initialize grand totals

    for page in dct.keys():                                         #loop through pages of earnings report
        for tb in dct[page].keys():                                 #loop through the text boxes on this page
            names    = dct[page][tb]['text'].split('\n')            #split the names string into lines
            earnings = dct[page][tb]['earnings'].split('\n')        #split the earnings string into lines
            
            for i in np.arange(len(names)):                         #loop through names
                if (len(names[i]) > 1):                             #
                    earnamt = float(earnings[i])                    #convert the corresponding earnings to float
                    total += earnamt                                #add to running total
                
    print(round(total,2))                                           #print the grand total

    return

### Salary matrix

In [ ]:
#start with 2015-2016 salary matrix

npcba_1516 =  np.array([[41286, 42900, 43871, 44505, 44893, 45186],
                        [44871, 46484, 47454, 48085, 48474, 48771],
                        [48494, 50106, 51078, 51709, 52098, 52393],
                        [52118, 53729, 54700, 55332, 55722, 56018],
                        [55743, 57354, 58328, 58958, 59347, 59642],
                        [59366, 60979, 61951, 62583, 62974, 63266],
                        [62991, 64605, 65574, 66206, 66596, 66892],
                        [66616, 68228, 69199, 69829, 69806, 70515],
                        [71741, 73353, 74323, 74954, 75345, 75639],
                        [78898, 80675, 81743, 82438, 82866, 83190]])
npcba_1415 = np.around(npcba_1516/1.02,0)                          #divide by 1.02 to get 2014-2015
npcba_1314 = np.around(npcba_1516/(1.025*1.02),0)                  #divide by 1.02*1.025 for 2013-2014
npcba_1213 = np.around((npcba_1516/(1.025*1.02*1.01)),0)           #divide by 1.02*1.025*1.01 for steps 1-9 2012-2013
npcba_1213[9,:] = np.around(npcba_1516[9,:]/(1.025*1.02*1.0225))   #divide by 1.02$1.025*1.0225 for step 10 2012-2013
npcba_1617 = npcba_1516                                            #2016-2017 is the same as 2015-2016
npcba_1718 = np.around(1.02*npcba_1516,0)                          #multiply 2015-2016 by 1.02 to get 2017-2018
npcba_1819 = np.around(1.0225*1.025*npcba_1516,0)                  #muntiply 2015-2016 by 1.02*1.0225 to get 2018-2019 
npcba_1920 = np.around(1.0225*1.025*npcba_1516,0)                  #2019-2020 same as 2018-2019 
npcba_2021 = np.around(1.02*1.0225*1.025*npcba_1516,0)             #2020-2021 2% increase
npcba_2122 = np.around(1.0225*1.02*1.0225*1.025*npcba_1516,0)      #2021-2022 2.25% increase 

cba = np.zeros((10, 10, 6))                                         #combine into 3-D numpy array
cba[9,:,:] = npcba_1213
cba[8,:,:] = npcba_1314
cba[7,:,:] = npcba_1415
cba[6,:,:] = npcba_1516
cba[5,:,:] = npcba_1617
cba[4,:,:] = npcba_1718
cba[3,:,:] = npcba_1819
cba[2,:,:] = npcba_1920
cba[1,:,:] = npcba_2021
cba[0,:,:] = npcba_2122

### Lookup function:  find step given salary 

In [ ]:
def get_step(rate,salary_matrix):                                           #determine step value
    cats = ['B','B+30','M','M+30','CAGS/M2','D']                            #categories
    years = ['2022','2021','2020','2019','2018','2017','2016']              #years
    salary = round(184.0*rate,0)                                            #salary is 184 times daily rate
    lower = salary - 1.0                                                    #lower limit for tolerance
    upper = salary + 1.0                                                    #upper limit for tolerance
    for i in np.arange(3,6):                                                #search salary matrix for a match
        for j in np.arange(10):
            for k in np.arange(6):
                if((lower <= salary_matrix[i,j,k]) & \
                   (salary_matrix[i,j,k] <= upper)):                        #computed salary within $1 of CBA
                    step = str(years[i]) + '-' + cats[k] + '-' + str(j+1)   #code is:  yyyy-cat-step
                    return(step)
    return('')

### Determine FTE value

In [ ]:
def get_fte(rate,earnings):
    fte = np.NaN
    fractions = [1/2.,1/3.,2/3.,1/4.,3/4.,1/5.,2/5.,3/5.,4/5.,1/6.,5/6.,\
                 1/7.,2/7.,3/7.,4/7.,5/7.,6/7.,1/8.,3/8.,5/8.,7/8.,\
                1/9.,2/9.,4/9.,5/9.,7/9.,8/9.,1/10.,3/10.,7/10.,9/10.,1.0]
    payments  = [21.,26.]
    min_abs_diff = 100000.
    
    if ((earnings > 150.) & (earnings < 4500.)):
        for paymts in payments:
            for frac in fractions:
                diff = abs(earnings - frac*rate*184/paymts)
                if (diff < min_abs_diff):
                    min_abs_diff = diff
                    fte = frac
    return(fte)

### Determine number of payments

In [ ]:
def get_number_of_payments(rate,earnings):
    number_of_payments = np.NaN
    fractions = [1/2.,1/3.,2/3.,1/4.,3/4.,1/5.,2/5.,3/5.,4/5.,1/6.,5/6.,\
                 1/7.,2/7.,3/7.,4/7.,5/7.,6/7.,1/8.,3/8.,5/8.,7/8.,\
                1/9.,2/9.,4/9.,5/9.,7/9.,8/9.,1/10.,3/10.,7/10.,9/10.,1.0]
    payments  = [21.,26.]
    min_abs_diff = 100000.
    
    if ((earnings > 150.) & (earnings < 4500.)):
        for paymts in payments:
            for frac in fractions:
                diff = abs(earnings - frac*rate*184/paymts)
                if (diff < min_abs_diff):
                    min_abs_diff = diff
                    number_of_payments = paymts 
    return(number_of_payments)

### Build a dictionary of payroll dates from FY2010 to FY2026

In [ ]:
def get_payperiods():
    
    payperiods = {}                                      #initialize payperiods dictionary

    delta = timedelta(days=14)                           #14 days per pay period
    x = date(2009,7,3)                                   #first pay period of FY2010
    fiscal_year = 2010-1                                 #initialize fiscal year
    last_month=6                                         #so that it will update to 2010 at the start

    while (x < date(2026,7,1)):                          #generate pay periods through the end of FY2026
        if ((x.month == 7) & (last_month == 6)):         #test for new fiscal year
            fiscal_year += 1                             #  increment fiscal year
            count = 0                                    #  set payperiod count to zero
        fseq = 1 + count % 26                            #  compute payperiod sequence number
        if (fiscal_year not in payperiods.keys()):       #check whether fiscal year is in dictionary yet
            payperiods[fiscal_year] = {}                 #if not, add it
        payperiods[fiscal_year][fseq] = x                #add payperiod to dictionary
        count += 1                                       #increment the payperiod count
        last_month = x.month                             #save current month to detect fiscal year end
        x += delta                                       #increment date by 14 days
        
    return(payperiods)

### Assign dates where possible

In [ ]:
def assign_dates(rsal,pdays):
    
    fyears = {}
    
    for ix in rsal.keys():                #build a dictionary with entries for each fiscal year
        fyear = rsal[ix]['fyear']
        if fyear not in fyears.keys():
            fyears[fyear] = {}
        acct = rsal[ix]['acct']
        if (acct not in fyears[fyear].keys()):
            fyears[fyear][acct] = {}
            fyears[fyear][acct]['min_payments'] = 26
            fyears[fyear][acct]['max_payments'] = 0
            fyears[fyear][acct]['n_payments']   = 0
            fyears[fyear][acct]['payments'] = {}
        if 'payments' in rsal[ix].keys():
            payments = rsal[ix]['payments']
        if (payments == payments):
            if (payments < fyears[fyear][acct]['min_payments']):
                fyears[fyear][acct]['min_payments'] = rsal[ix]['payments']
            if (payments > fyears[fyear][acct]['max_payments']):
                fyears[fyear][acct]['max_payments'] = rsal[ix]['payments']
        fyears[fyear][acct]['n_payments'] += 1
        fyears[fyear][acct]['payments'][ix] = {}

    for fyear in fyears.keys():
        for acct in fyears[fyear].keys():
            if ((fyears[fyear][acct]['min_payments'] == fyears[fyear][acct]['max_payments'])):    #no change in payments
                n_payments = fyears[fyear][acct]['n_payments']
                if (n_payments==26.0):
                    dix = 1
                    for nix in fyears[fyear][acct]['payments'].keys():
                        fyears[fyear][acct]['payments'][nix]['date'] = paydays[fyear][dix]
                        dix += 1
                elif (n_payments ==23.0):
                    dix = 4
                    for nix in fyears[fyear][acct]['payments'].keys():
                        fyears[fyear][acct]['payments'][nix]['date'] = paydays[fyear][dix]
                        dix += 1
                elif (n_payments ==21.0):
                    dix = 4
                    for nix in fyears[fyear][acct]['payments'].keys():
                        fyears[fyear][acct]['payments'][nix]['date'] = paydays[fyear][dix]
                        dix += 1
                elif (fyears[fyear][acct]['n_payments']==3.0):
                    dix = 1
                    for nix in fyears[fyear][acct]['payments'].keys():
                        fyears[fyear][acct]['payments'][nix]['date'] = paydays[fyear][dix]
                        dix += 1
    return(fyears)

### Get dictionary of paydays: FY2010-FY2023

In [ ]:
paydays = get_payperiods()

### Read the FY2017 earnings report

In [ ]:
p17 = read_earnings('../FY17 Gene_Redacted.pdf')

### Read the FY2018 earnings report

In [ ]:
p18 = read_earnings('../FY18 Gene_Redacted.pdf')

### Process the FY17 Earnings report

In [ ]:
dnames17 = get_names(p17)                                   #extract name records from pdf
consolidated_names17 = consolidate_name_boxes(dnames17)     #consolidate overlapping text boxes containing names
get_matches(consolidated_names17,p17)                       #match text boxes for other columns by vertical pos

### Process the FY18 Earnings report

In [ ]:
dnames18 = get_names(p18)                                   #extract name records from pdf
consolidated_names18 = consolidate_name_boxes(dnames18)     #consolidate overlapping text boxes containing names
get_matches(consolidated_names18,p18)                       #match text boxes for other columns by vertical pos

### Check sum of earnings against total on earnings reports

In [ ]:
balancing_totals(consolidated_names17)     #FY2017 earnings report total is $22,608,024.34

In [ ]:
balancing_totals(consolidated_names18)     #FY2018 earnings report total is $22,409,915.41

### Get dictionary of people

In [ ]:
people = {}

ay = {}                                                           #combine FY17 and FY18
ay[2017] = consolidated_names17
ay[2018] = consolidated_names18
                                                                  #build people dictionary for all years 
for year in ay.keys():                                            # loop through years
    for page in ay[year].keys():                                  # loop through pages of earnings report
        for tb in ay[year][page].keys():                          # loop through text boxes on each page
            fund = ay[year][page][tb]['fund']
            acct = ay[year][page][tb]['acct']
            obj  = ay[year][page][tb]['obj']
            
            txt  = ay[year][page][tb]['text']                      # pick up names
            positions = ay[year][page][tb]['positions']
            rates     = ay[year][page][tb]['rates']
            earnings  = ay[year][page][tb]['earnings']
            
            lines  = txt.split('\n')                               # split names string into lines
            plines = positions.split('\n')
            rlines = rates.split('\n')
            elines = earnings.split('\n')
            
            for i in np.arange(len(lines)):                       # add name to people dictionary
                if (len(lines[i]) > 1):
                    name = lines[i]
                    if (name not in people.keys()):
                        people[name] = {}
                    if (fund not in people[name].keys()):
                        people[name][fund] = {}
                    if (acct not in people[name][fund].keys()):
                        people[name][fund][acct] = {}
                    if (obj not in people[name][fund][acct].keys()):
                        people[name][fund][acct][obj] = {}
                        
                    seq = 1+len(people[name][fund][acct][obj].keys())
                    
                    people[name][fund][acct][obj][seq] = {}
                    people[name][fund][acct][obj][seq]['position'] = plines[i]
                    rate = float(rlines[i])
                    people[name][fund][acct][obj][seq]['rate'] = rate
                    earnings = float(elines[i])
                    people[name][fund][acct][obj][seq]['earnings'] = earnings
                    people[name][fund][acct][obj][seq]['fyear'] = year
                    if (rate > 200.0):
                        step = get_step(rate,cba)
                        people[name][fund][acct][obj][seq]['step'] = step
                        fte = get_fte(rate,earnings)
                        people[name][fund][acct][obj][seq]['fte'] = fte
                        payments = get_number_of_payments(rate,earnings)
                        people[name][fund][acct][obj][seq]['payments'] = payments

### Summarize regular salary for entries on CBA salary matrix

In [ ]:
regsal = {}                                                   #dictionary for summary by name

for name in people.keys():                                    #loop through people by name
    for fund in people[name].keys():                          #loop through fund, acct-code
        for acct in people[name][fund].keys():
            for obj in people[name][fund][acct].keys():       #select only obj=51110 regular salary
                if (obj == '51110'):
                    for seq in people[name][fund][acct][obj].keys():                 #select only if step coded
                        if ('step' in people[name][fund][acct][obj][seq].keys()):    #add to summary dictionary
                            if (name not in regsal.keys()):
                                regsal[name] = {}
                            rseq = 1+len(regsal[name].keys())
                            regsal[name][rseq] = {}
                            regsal[name][rseq]['fyear'] = people[name][fund][acct][obj][seq]['fyear']
                            regsal[name][rseq]['acct']  = acct
                            regsal[name][rseq]['step']  = people[name][fund][acct][obj][seq]['step']
                            regsal[name][rseq]['fte']  = people[name][fund][acct][obj][seq]['fte']
                            regsal[name][rseq]['payments']  = people[name][fund][acct][obj][seq]['payments']
                            regsal[name][rseq]['earnings'] = people[name][fund][acct][obj][seq]['earnings']

### Add dates

In [ ]:
for name in regsal.keys():
    pdates = assign_dates(regsal[name],paydays)
    for fyear in pdates.keys():
        for acct in pdates[fyear].keys():
            for ix in pdates[fyear][acct]['payments'].keys():
                try:
                    regsal[name][ix]['date'] = pdates[fyear][acct]['payments'][ix]['date']
                except KeyError:
                    regsal[name][ix]['date'] = np.NaN

### Print results for those with step values

In [ ]:
print("Names with step values: ",len(regsal))         #show the number of distinct names

for name in regsal.keys():                            #compute totals by fiscal year and percent increase
    totsal = {}
    print('\n',name)                                  #start of data for this employee
    for seq in regsal[name].keys():                   #loop through earnings entries and add them up by fyear
        fyear = regsal[name][seq]['fyear']
        if (fyear not in totsal.keys()):
            totsal[fyear] = 0.0
        acct  = regsal[name][seq]['acct'] 
        try:
            acct_desc = accd[acct]
        except KeyError:
            acct_desc = '(no label)'
        step  = regsal[name][seq]['step']
        fte  = round(regsal[name][seq]['fte'],1)
        payments  = regsal[name][seq]['payments']
        earnings = regsal[name][seq]['earnings']
        try:
            pdate = regsal[name][seq]['date']
        except KeyError:
            pdate = '(no date)'
        totsal[fyear]+= earnings
        print(seq,fyear,acct,acct_desc,pdate,fte,step, payments,earnings)
    totstr = ''
    for fy in totsal.keys():
        totstr += str(fy) + ' ' + str(round(totsal[fy],2)) + ' '
    pct_increase = ' '
    if(len(totsal)==2):                             #if more than one year, compute percent change                              
        pct_increase = '  percent change: ' +\
        str(round(100*((totsal[2018]/totsal[2017])-1.0),2))
    totstr = totstr + pct_increase
    print('\n',name,' total regular salary: ',totstr)